Allen Hoskins 

DS7337: Natural Language Processing, Fall 2022

# <center> <u>Homework 5</u>

In [5]:
import numpy as np
import pandas as pd
import nltk
import requests
from bs4 import BeautifulSoup

## 1. Compile a list of static links (permalinks) to individual user movie reviews from one particular website. 

This will be your working dataset for this assignment, as well as for assignments 7 and 8.   

a. It does not matter if you use a crawler or if you manually collect the links, but you will need at least 100 movie review links. Note that, as of this writing, the robots.txt file of IMDB.com allows the crawling of user reviews.

b. Each link should be to a web page that has only one user review of only one movie, e.g., the user review permalinks on the IMDB site.

c.	Choose reviews of movies that are all in the same genre, e.g., sci-fi, mystery, romance, superhero, etc.  

d.	Make sure your collection includes reviews of several movies in your chosen genre and that it includes a mix of negative and positive reviews.  

In [6]:
def get_permalinks(reviews_url):
    
    response = requests.get(reviews_url)
    
    soup = BeautifulSoup(response.content, "html.parser")
    
    a_tags = soup.find_all("a")
    
    permalinks = []
    
    for tag in a_tags:
        if tag.get_text() == "Permalink":
            permalink = "https://www.imdb.com" + tag.attrs['href']
            permalinks.append(permalink)
            
    return permalinks

def get_all_permalinks(pages_dict):
    
    all_permalinks = []
    
    for movie_name, review_page_link in pages_dict.items():
        
        permalinks = get_permalinks(reviews_url=review_page_link)
        all_permalinks += permalinks
        
        title = ' '.join(movie_name.split("_")).title()
        print(f"Scraped {len(permalinks)} Review Permalinks for {title}")
    
    return all_permalinks

In [7]:
review_pages = {'thor:_love_and_thunder':"https://www.imdb.com/title/tt10648342/reviews/?ref_=tt_ql_urv", 
                'dc_league_of_super_pets':"https://www.imdb.com/title/tt8912936/reviews/?ref_=tt_ql_urv", 
                "everything_everywhere_all_at_once":"https://www.imdb.com/title/tt6710474/reviews/?ref_=tt_ql_urv", 
                "the_lost_city":"https://www.imdb.com/title/tt13320622/reviews/?ref_=tt_ql_urv", 
                "deadpool_2":"https://www.imdb.com/title/tt5463162/reviews/?ref_=tt_ql_urv", 
                "deadpool":"https://www.imdb.com/title/tt1431045/reviews/?ref_=tt_ql_urv", 
                "land_of_the_lost":"https://www.imdb.com/title/tt0457400/reviews/?ref_=tt_ql_urv", 
                "free_guy":"https://www.imdb.com/title/tt6264654/reviews/?ref_=tt_ql_urv", 
                "the_suicide_squad":"https://www.imdb.com/title/tt6334354/reviews/?ref_=tt_ql_urv",
                "guardians_of_the_galaxy":"https://www.imdb.com/title/tt2015381/reviews/?ref_=tt_ql_urv"}

plinks = get_all_permalinks(pages_dict=review_pages)

Scraped 25 Review Permalinks for Thor: Love And Thunder
Scraped 25 Review Permalinks for Dc League Of Super Pets
Scraped 25 Review Permalinks for Everything Everywhere All At Once
Scraped 25 Review Permalinks for The Lost City
Scraped 24 Review Permalinks for Deadpool 2
Scraped 25 Review Permalinks for Deadpool
Scraped 25 Review Permalinks for Land Of The Lost
Scraped 25 Review Permalinks for Free Guy
Scraped 25 Review Permalinks for The Suicide Squad
Scraped 24 Review Permalinks for Guardians Of The Galaxy


In [8]:
plinks 

['https://www.imdb.com/review/rw8310329/',
 'https://www.imdb.com/review/rw8309971/',
 'https://www.imdb.com/review/rw8351608/',
 'https://www.imdb.com/review/rw8315692/',
 'https://www.imdb.com/review/rw8334081/',
 'https://www.imdb.com/review/rw8313357/',
 'https://www.imdb.com/review/rw8323305/',
 'https://www.imdb.com/review/rw8329077/',
 'https://www.imdb.com/review/rw8318859/',
 'https://www.imdb.com/review/rw8315634/',
 'https://www.imdb.com/review/rw8332897/',
 'https://www.imdb.com/review/rw8311620/',
 'https://www.imdb.com/review/rw8310746/',
 'https://www.imdb.com/review/rw8313711/',
 'https://www.imdb.com/review/rw8314433/',
 'https://www.imdb.com/review/rw8304194/',
 'https://www.imdb.com/review/rw8306334/',
 'https://www.imdb.com/review/rw8307205/',
 'https://www.imdb.com/review/rw8312037/',
 'https://www.imdb.com/review/rw8331400/',
 'https://www.imdb.com/review/rw8525634/',
 'https://www.imdb.com/review/rw8312102/',
 'https://www.imdb.com/review/rw8311274/',
 'https://w

## 2.	Extract noun phrase (NP) chunks from your reviews using the following procedure:

a. In Python, use BeautifulSoup to grab the main review text from each link.  

b.	Next run each review text through a tokenizer, and then try to NP-chunk it with a shallow parser. 

c.	You probably will have too many unknown words, owing to proper names of characters, actors, and so on that are not in your working dictionary. Make sure the main names that are relevant to the movies in your collection of reviews are added to the working lexicon, and then run the NP chunker again.

In [25]:
reviews = []
for url in plinks:
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser').find("div",class_='text show-more__control')
    text = soup.get_text()
    reviews.append(text)

In [38]:
from nltk import word_tokenize,pos_tag
from nltk.chunk import ne_chunk




In [43]:
grammar = "NP: {<DT>?<JJ>*<NN>}"
NPChunker = nltk.RegexpParser(grammar)
result = NPChunker.parse(reviews[0])
result.draw()

TypeError: Tree() argument 2 should be a list, not a string

In [54]:
import nltk
import pprint
import re
import tkinter



ModuleNotFoundError: No module named '_tkinter'

In [51]:
output = chunker.parse(pos_tag(word_tokenize(reviews[0])))
print("After Chunking",output)
output.draw()

After Chunking (S
  (mychunk Thor/NN)
  :/:
  (mychunk Love/NNP and/CC)
  (mychunk Thunder/NNP)
  does/VBZ
  attempt/VB
  to/TO
  explore/VB
  (mychunk themes/NNS)
  of/IN
  (mychunk love/NN and/CC)
  (mychunk loss/NN whilst/NN)
  introducing/VBG
  the/DT
  (mychunk Mighty/NNP Thor/NNP and/CC)
  putting/VBG
  (mychunk Thor/NNP)
  on/IN
  a/DT
  (mychunk journey/NN)
  of/IN
  (mychunk self/JJ)
  (mychunk discovery/NN)
  ./.
  However/RB
  ,/,
  it/PRP
  sadly/RB
  does/VBZ
  n't/RB
  work/VB
  as/RB
  well/RB
  as/IN
  it/PRP
  should/MD
  due/VB
  to/TO
  a/DT
  (mychunk rushed/JJ)
  (mychunk pace/NN and/CC)
  (mychunk way/NN)
  too/RB
  (mychunk many/JJ)
  (mychunk jokes/NNS)
  that/WDT
  are/VBP
  almost/RB
  never/RB
  (mychunk funny.Chris/JJ)
  (mychunk Hemsworth/NNP)
  is/VBZ
  still/RB
  going/VBG
  (mychunk strong/JJ)
  as/IN
  (mychunk Thor/NN but/CC)
  the/DT
  (mychunk extreme/JJ)
  (mychunk goofiness/NN)
  is/VBZ
  getting/VBG
  a/DT
  (mychunk little/JJ)
  (mychunk stale/NN

ModuleNotFoundError: No module named '_tkinter'

In [37]:
full_chunks = []
for review in reviews:
    chunks = ne_chunk(pos_tag(word_tokenize(review)))
    full_chunks.append([chunks])

In [35]:
from sklearn import tree


print(tree)

[Tree('S', [Tree('GPE', [('Thor', 'NN')]), (':', ':'), Tree('PERSON', [('Love', 'NNP')]), ('and', 'CC'), Tree('PERSON', [('Thunder', 'NNP')]), ('does', 'VBZ'), ('attempt', 'VB'), ('to', 'TO'), ('explore', 'VB'), ('themes', 'NNS'), ('of', 'IN'), ('love', 'NN'), ('and', 'CC'), ('loss', 'NN'), ('whilst', 'NN'), ('introducing', 'VBG'), ('the', 'DT'), Tree('ORGANIZATION', [('Mighty', 'NNP'), ('Thor', 'NNP')]), ('and', 'CC'), ('putting', 'VBG'), Tree('PERSON', [('Thor', 'NNP')]), ('on', 'IN'), ('a', 'DT'), ('journey', 'NN'), ('of', 'IN'), ('self', 'JJ'), ('discovery', 'NN'), ('.', '.'), ('However', 'RB'), (',', ','), ('it', 'PRP'), ('sadly', 'RB'), ('does', 'VBZ'), ("n't", 'RB'), ('work', 'VB'), ('as', 'RB'), ('well', 'RB'), ('as', 'IN'), ('it', 'PRP'), ('should', 'MD'), ('due', 'VB'), ('to', 'TO'), ('a', 'DT'), ('rushed', 'JJ'), ('pace', 'NN'), ('and', 'CC'), ('way', 'NN'), ('too', 'RB'), ('many', 'JJ'), ('jokes', 'NNS'), ('that', 'WDT'), ('are', 'VBP'), ('almost', 'RB'), ('never', 'RB'), (

## 3. Output all the chunks in a single list for each review 

submit that output for this assignment. Also submit a brief written summary of what you did (describe your selection of genre, your source of reviews, how many you collected, and by what means).
